Two-link Planar Arm:

Length of each link ($l$) = 0.5

action: $[w1, w2]$ <br />
$w1$ = joint1 angular velocity <br />
$w2$ = joint2 angular velocity

state: $[t1, t2]$ <br />
$t1$ = joint1 angle <br />
$t2$ = joint2 angle <br />

position of end effector: <br />
$x$ = x position <br />
$y$ = y position <br />

In [26]:
import torch as t
import math
import numpy as np

state=t.tensor([2*math.pi,2*math.pi,1,1])*(t.rand(3,4)-0.5)
print(state)

action=t.rand(3,2)
print(action)




tensor([[-0.4523, -0.2284, -0.4701, -0.4698],
        [-2.5880,  3.1304,  0.1419,  0.2314],
        [ 2.7630, -1.6823, -0.0975, -0.1287]])
tensor([[0.1048, 0.7837],
        [0.2448, 0.7325],
        [0.9693, 0.9475]])
